In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import multivariate_normal

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/sales-forecast-et5003-competition/train.csv')
train.iloc[5:10,:]

In [ ]:
X = np.linspace(0,1,100)[:,None]
Y = train.iloc[18,1:101].values[:,None]
plt.plot(X, Y)

In [ ]:
X = np.linspace(0,1,100)
Y =  X/2-2*np.exp(-(X-0.5)**2) + 2 + np.random.randn(len(X))*0.05
X = X[:,None]
Y = Y[:, None]
plt.plot(X,Y)

In [ ]:
with pm.Model() as model:
    #  periodic component x 
#     η_per = pm.HalfCauchy("η_per", beta=2, testval=1.0)
#     period  = pm.Normal("period", mu=1, sigma=3)
#     ℓ_psmooth = pm.Gamma("ℓ_psmooth ", alpha=1, beta=1)
    
    η_true  = pm.Normal("η_true", mu=1, sigma=3)
    ℓ_true  = pm.Normal("ℓ_true", mu=1, sigma=3)

    
#     cov_seasonal = η_per**2 * pm.gp.cov.Periodic(1, period, ℓ_psmooth)  
    cov_seasonal = η_true**2 * pm.gp.cov.ExpQuad(1, ℓ_true)
    gp1 = pm.gp.Marginal(cov_func=cov_seasonal)
    
    #  linear component x 
#     tau = pm.HalfCauchy("tau", beta=2, testval=1.0)
#     c  = pm.Normal("c", mu=0, sigma=1) 
    
#     cov_linear = tau * pm.gp.cov.Linear(1, c)  
    cov_linear = η_true**2 * pm.gp.cov.Matern32(1, ℓ_true)
    gp2 = pm.gp.Marginal(cov_func=cov_linear)
    
    # gp represents f1 + f2.
    gp = gp1 + gp2
    
    # noise model
    sigma = pm.HalfCauchy("sigma", beta=5, testval=0.1)
    f = gp.marginal_likelihood("f", X, Y[:,0], noise=sigma)
    
    # this line calls an optimizer to optimize the marginal likelihood
    mp = pm.find_MAP(include_transformed=True)

In [ ]:
#optimized parameters
sorted([name+":"+str(mp[name]) for name in mp.keys() if not name.endswith("_")])

In [ ]:
X_new = np.linspace(0, 1, 100)[:,None]

#periodic term
f1_pred = gp1.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma})
y1_pred = gp1.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma}, pred_noise=True)
#linear term
f2_pred = gp2.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma})
y2_pred = gp2.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma}, pred_noise=True)
#sum of periodic and linear term
y_pred  = gp.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma}, pred_noise=True)
f_pred  = gp.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma})


#we sample 10 regression lines from the posterior
sample_f = multivariate_normal(f_pred[0],f_pred[1]).rvs(50)

plt.plot(X_new[:,0],sample_f.T,color='red',alpha=0.1)# regression lines
plt.plot(X_new[:,0],f_pred[0])# this is the average line
plt.plot(X, Y, 'og', ms=3, alpha=0.9, label="Observed data", Zorder=1000);

plt.figure()
#we sample 10 regression lines from the posterior
sample_y = multivariate_normal(y_pred[0],y_pred[1]).rvs(50)

plt.plot(X_new[:,0],sample_y.T,color='red',alpha=0.1)# regression lines
plt.plot(X_new[:,0],f_pred[0])# this is the average line
plt.plot(X, Y, 'og', ms=3, alpha=0.9, label="Observed data", Zorder=1000);

In [ ]:
#we can recover the single components
plt.plot(X_new[:,0],f1_pred[0])
plt.plot(X_new[:,0],f2_pred[0])
#plt.plot(X, y, 'or', ms=3, alpha=0.9, label="Observed data");

In [ ]:
# f2_pred[1][2]
plt.plot(X_new[:,0],f2_pred[1][4])

In [ ]:
Gradients = []
for i in range(f2_pred[0].shape[0]):
    Gradients.append(np.min(np.gradient(f2_pred[0][i,:], X)))


In [ ]:
# f2_pred[0].shape
#  X.reshape(100,)
X.shape

In [ ]:
# np.gradient(f2_pred[0])
# np.gradient()
# len(np.where(np.gradient(f2_pred[0],X_new[:,0])<0)[0])
len(f2_pred[0])

In [ ]:
# np.mean(f2_pred[1],axis=0)
f2_pred[0]

In [ ]:
def run_model(X,Y,plot=False):    
    
    with pm.Model() as model:
        #  periodic component x 
#         ℓ_true = pm.HalfCauchy("η_per", beta=2, testval=1.0)
        η_true  = pm.Normal("η_true", mu=0.1, sigma=3)
        ℓ_true  = pm.Normal("ℓ_true", mu=0.1, sigma=3)
#         ℓ_psmooth = pm.Gamma("ℓ_psmooth ", alpha=1, beta=1)

        cov_seasonal = η_true**2 * pm.gp.cov.ExpQuad(1, ℓ_true) # Maybe change this to 100 since its 100 dimentions   
#         cov_seasonal = η_per**2 * pm.gp.cov.Periodic(1, period, ℓ_psmooth)
        gp1 = pm.gp.Marginal(cov_func=cov_seasonal)

        #  linear component x 
#         tau = pm.HalfCauchy("tau", beta=2, testval=1.0)
#         c  = pm.Normal("c", mu=0, sigma=1) 

#         cov_linear = tau * pm.gp.cov.Linear(1, c)  
        cov_linear = η_true**2 * pm.gp.cov.Matern32(1, ℓ_true)
        gp2 = pm.gp.Marginal(cov_func=cov_linear)

        # gp represents f1 + f2.
        gp = gp1 + gp2

        # noise model
        sigma = pm.HalfCauchy("sigma", beta=5, testval=0.1)
        f = gp.marginal_likelihood("f", X, Y[:,0], noise=sigma)

        # this line calls an optimizer to optimize the marginal likelihood
        mp = pm.find_MAP(include_transformed=True)
        #we can do  an approximated inference
#     with model:
#         inference = pm.ADVI()
#         approx = pm.fit(60000, method=inference)
        
#     posterior = approx.sample(draws=500)
    
#     all_prediction = np.dot(H,posterior['weights'].T).T
#     non_periodic_prediction = np.dot(H_np,posterior['weights'][:,0:col_per].T).T
    X_new = np.linspace(0, 1, 100)[:,None]

    #periodic term
    f1_pred = gp1.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma})
    y1_pred = gp1.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma}, pred_noise=True)
    #linear term
    f2_pred = gp2.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma})
    y2_pred = gp2.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma}, pred_noise=True)
    #sum of periodic and linear term
    y_pred  = gp.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma}, pred_noise=True)
    f_pred  = gp.predict(X_new, point=mp, given={"gp": gp, "X": X, "y": Y[:,0], "noise": sigma})

    if plot==True:
        plt.figure()
        plt.plot(x,np.mean(f_pred,axis=0),'r', label='Overall Mean')
        plt.plot(x,np.mean(f2_pred,axis=0),'b', label='Mean of the non-periodic comp.')
        plt.legend()
        plt.scatter(X,Y)
#     Gradients = []
#     for i in range(f2_pred[0].shape[0]):
#         Gradients.append(np.min(np.gradient(f2_pred[1][i,:], X_new[:,0])))
    
#     posterior_probability_deriviative_is_positive = np.min(np.gradient(f2_pred[0], X_new[:,0]))
    posterior_probability_deriviative_is_positive = len(np.where(np.gradient(f2_pred[0],X_new[:,0])>0)[0])/len(f2_pred[0])
    print("probability that the function is increasing=", posterior_probability_deriviative_is_positive)
    if posterior_probability_deriviative_is_positive>0.8:
        return 1
    else:
        return 0


In [ ]:
# test_df = train.iloc[5:10,:]
# test_df
test_df= pd.read_csv('/kaggle/input/sales-forecast-et5003-competition/test.csv')
test_df.head()

In [ ]:
x = np.linspace(0,1,100)[:,None]
Decision = pd.DataFrame(columns=['Id','Category'])
for r in range(test_df.shape[0]):
    id_row = test_df.iloc[r,0]
    y = test_df.iloc[r,1:101].values[:,None]
    decision = run_model(x,y)
    Decision = Decision.append({'Id': int(id_row), 'Category': int(decision)}, ignore_index=True) 
#     print(Decision)

Decision.to_csv('submission_1.csv')